In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [7]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def get_positional_dbs(circuit, circuit_db):

    qubits_involved = get_qubits_involved(circuit, circuit_db)
    
    gates_on_qubit = {q:[] for q in qubits_involved}
    on_qubit_order = {q:[] for q in qubits_involved}

    for order_gate, ind_gate in enumerate( circuit_db["ind"]):
        if ind_gate < translator.number_of_cnots:
            control, target = translator.indexed_cnots[str(ind_gate)]
            gates_on_qubit[control].append(ind_gate)
            gates_on_qubit[target].append(ind_gate)
            on_qubit_order[control].append(order_gate)                
            on_qubit_order[target].append(order_gate)  
        else:
            gates_on_qubit[(ind_gate-translator.n_qubits)%translator.n_qubits].append(ind_gate)
            on_qubit_order[(ind_gate-translator.n_qubits)%translator.n_qubits].append(order_gate)        
    return gates_on_qubit, on_qubit_order

In [155]:
translator = CirqTranslater(1)
db1 = u1_layer(translator)
circuit_db = concatenate_dbs([db1]*2)
circuit, circuit_db  = translator.give_circuit(circuit_db, unresolved=False)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()


simplification = False
for q, qubit_gates_path in gates_on_qubit.items():
    if simplification is True:
        break
    for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-2]):
        if simplification is True:
            break
        ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
        ind_gate_p2 = qubit_gates_path[order_gate_on_qubit+2]
        check_rot = lambda ind_gate: translator.number_of_cnots<= ind_gate <(3*translator.n_qubits + translator.number_of_cnots)

        if (check_rot(ind_gate) == True) and (check_rot(ind_gate_p1) == True) and (check_rot(ind_gate_p2) == True):

            type_get = lambda x: (x-translator.number_of_cnots)//translator.n_qubits
            type_0 = type_get(ind_gate)
            type_1 = type_get(ind_gate_p1)
            type_2 = type_get(ind_gate_p2)


            if type_0 == type_2:
                types = [type_0, type_1, type_2]
                for next_order_gate_on_qubit, ind_gate_next in enumerate(qubit_gates_path[order_gate_on_qubit+3:]):
                    print(q, ind_gate, ind_gate_next)
                    if (check_rot(ind_gate_next) == True):# and (next_order_gate_on_qubit < len(qubit_gates_path[order_gate_on_qubit+3:])):
                        types.append(type_get(ind_gate_next))
                        simplification=True                        
                    else:
                        break
                if simplification == True:
                    translator.translator_ = CirqTranslater(n_qubits=2)
                    u_to_compile_db = simplified_db.loc[indices_to_compile]
                
                    u_to_compile_db["ind"] = translator.translator_.n_qubits*(u_to_compile_db["ind"] - translator.number_of_cnots)//translator.n_qubits    + translator.translator_.number_of_cnots
                    u_to_compile_db["symbol"] = None ##just to be sure it makes no interference with the compiler...

                    
                    compile_circuit, compile_circuit_db = construct_compiling_circuit(translator.translator_, u_to_compile_db)
                    minimizer = Minimizer(translator.translator_, mode="compiling", hamiltonian="Z")

                    cost, resolver, history = minimizer.minimize([compile_circuit], symbols=translator.get_symbols(compile_circuit_db))

                    
                    u1s = u1_db(translator, 0, params=True)
                    u1s["param_value"] = -np.array(list(resolver.values()))
                    resu_comp, resu_db = translator.give_circuit(u1s,unresolved=False)
                    
                    cc, cdb = translator.translator_.give_circuit(u_to_compile_db, unresolved=False)
                    c = cc.unitary()
                    r = resu_comp.unitary()
                    
                    ## phase_shift if necessary
                    if np.abs(np.mean(c/r) -1) > 1:
                        u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + 2*np.pi)# Rz(\th) = e^{-ii \theta \sigma_z / 2}c0, cdb0 = translator.give_circuit(pd.DataFrame([gate_template(0, param_value=2*np.pi)]), unresolved=False)
                    resu_comp, resu_db = translator.give_circuit(u1s,unresolved=False)

                    
                    for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
                        simplified_db.loc[new_ind+0.1] = gate_template(translator.number_of_cnots + q + typ*translator.n_qubits,
                                                                         param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"], 
                                                                         trainable=True)#, symbol=symbols[new_ind])
                    for old_inds in indices_to_compile:
                        simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#
                    simplified_db = simplified_db.sort_index().reset_index(drop=True)
                    

0 0 0
0 0 1
0 0 0


In [159]:
sc, scdb = translator.give_circuit(simplified_db,unresolved=False)
sc.unitary()

array([[-0.85055965-0.368102j  , -0.35882368+0.11088177j],
       [ 0.35882368+0.11088177j, -0.85055965+0.368102j  ]])

In [160]:
circuit.unitary()

array([[-0.85055767-0.36811656j, -0.35874677+0.11109724j],
       [ 0.35874677+0.11109724j, -0.85055767+0.36811656j]])

In [161]:
simplified_db

,ind,symbol,param_value,trainable,block_id
0,0,None,0.862651,True,0
1,1,None,7.053198,True,0
2,0,None,-14.245899,True,0


In [162]:
def rule_5(translator, simplified_db, on_qubit_order, gates_on_qubit):
    """
    compile 1-qubit gates into euler rotations
    """
    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-2]):
            if simplification is True:
                break
            ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
            ind_gate_p2 = qubit_gates_path[order_gate_on_qubit+2]
            check_rot = lambda ind_gate: translator.number_of_cnots<= ind_gate <(3*translator.n_qubits + translator.number_of_cnots)

            if (check_rot(ind_gate) == True) and (check_rot(ind_gate_p1) == True) and (check_rot(ind_gate_p2) == True):

                type_get = lambda x: (x-translator.number_of_cnots)//translator.n_qubits
                type_0 = type_get(ind_gate)
                type_1 = type_get(ind_gate_p1)
                type_2 = type_get(ind_gate_p2)


                if type_0 == type_2:
                    types = [type_0, type_1, type_2]
                    for next_order_gate_on_qubit, ind_gate_next in enumerate(qubit_gates_path[order_gate_on_qubit+3:]):
                        print(q, ind_gate, ind_gate_next)
                        if (check_rot(ind_gate_next) == True):# and (next_order_gate_on_qubit < len(qubit_gates_path[order_gate_on_qubit+3:])):
                            types.append(type_get(ind_gate_next))
                            simplification=True                        
                        else:
                            break
                    if simplification == True:
                        translator.translator_ = CirqTranslater(n_qubits=2)
                        u_to_compile_db = simplified_db.loc[indices_to_compile]

                        u_to_compile_db["ind"] = translator.translator_.n_qubits*(u_to_compile_db["ind"] - translator.number_of_cnots)//translator.n_qubits    + translator.translator_.number_of_cnots
                        u_to_compile_db["symbol"] = None ##just to be sure it makes no interference with the compiler...


                        compile_circuit, compile_circuit_db = construct_compiling_circuit(translator.translator_, u_to_compile_db)
                        minimizer = Minimizer(translator.translator_, mode="compiling", hamiltonian="Z")

                        cost, resolver, history = minimizer.minimize([compile_circuit], symbols=translator.get_symbols(compile_circuit_db))


                        u1s = u1_db(translator, 0, params=True)
                        u1s["param_value"] = -np.array(list(resolver.values()))
                        resu_comp, resu_db = translator.give_circuit(u1s,unresolved=False)

                        cc, cdb = translator.translator_.give_circuit(u_to_compile_db, unresolved=False)
                        c = cc.unitary()
                        r = resu_comp.unitary()

                        ## phase_shift if necessary
                        if np.abs(np.mean(c/r) -1) > 1:
                            u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + 2*np.pi)# Rz(\th) = e^{-ii \theta \sigma_z / 2}c0, cdb0 = translator.give_circuit(pd.DataFrame([gate_template(0, param_value=2*np.pi)]), unresolved=False)
                        resu_comp, resu_db = translator.give_circuit(u1s,unresolved=False)


                        for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
                            simplified_db.loc[new_ind+0.1] = gate_template(translator.number_of_cnots + q + typ*translator.n_qubits,
                                                                             param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"], 
                                                                             trainable=True)#, symbol=symbols[new_ind])
                        for old_inds in indices_to_compile:
                            simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#
                        simplified_db = simplified_db.sort_index().reset_index(drop=True)
        break                
    return simplification, simplified_db


def apply_rule(original_circuit_db, rule, **kwargs):
    max_cnt = kwargs.get('max_cnt',10)
    simplified, cnt = True, 0
    original_circuit, original_circuit_db = translator.give_circuit(original_circuit_db)
    gates_on_qubit, on_qubit_order = get_positional_dbs(original_circuit, original_circuit_db)
    simplified_db = original_circuit_db.copy()
    while simplified and cnt < max_cnt:
        simplified, simplified_circuit_db = rule(translator, simplified_db, on_qubit_order, gates_on_qubit)
        circuit, simplified_db = translator.give_circuit(simplified_circuit_db)
        gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, simplified_db)
        cnt+=1
    return cnt, simplified_db

In [163]:
translator = CirqTranslater(2)
db1 = u1_layer(translator)
circuit_db = concatenate_dbs([db1]*2)
circuit, circuit_db  = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()
simplification, ssimplified_db = rule_5(translator, simplified_db, on_qubit_order, gates_on_qubit)

0 2 2
0 2 4
0 2 2


ValueError: operands could not be broadcast together with shapes (4,4) (2,2) 

In [5]:
translator.give_circuit(circuit_db)[0]

(0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_6)───Rx(th_7)────Rz(th_8)────

(0, 1): ───Rz(th_3)───Rx(th_4)───Rz(th_5)───Rz(th_9)───Rx(th_10)───Rz(th_11)───

In [6]:
translator.give_circuit(ssimplified_db)[0]

(0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_6)───Rx(th_7)────Rz(th_8)────

(0, 1): ───Rz(th_3)───Rx(th_4)───Rz(th_5)───Rz(th_9)───Rx(th_10)───Rz(th_11)───

In [28]:
translator = CirqTranslater(1)
db1 = u1_layer(translator)
circuit_db = concatenate_dbs([db1]*2)
circuit, circuit_db  = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()


simplification = False
for q, qubit_gates_path in gates_on_qubit.items():
    if simplification is True:
        break
    for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-2]):
        if simplification is True:
            break
        ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
        ind_gate_p2 = qubit_gates_path[order_gate_on_qubit+2]
        check_rot = lambda ind_gate: translator.number_of_cnots<= ind_gate <(3*translator.n_qubits + translator.number_of_cnots)

        if (check_rot(ind_gate) == True) and (check_rot(ind_gate_p1) == True) and (check_rot(ind_gate_p2) == True):

            type_get = lambda x: (x-translator.number_of_cnots)//translator.n_qubits
            type_0 = type_get(ind_gate)
            type_1 = type_get(ind_gate_p1)
            type_2 = type_get(ind_gate_p2)


            if type_0 == type_2:
                types = [type_0, type_1, type_2]
                for next_order_gate_on_qubit, ind_gate_next in enumerate(qubit_gates_path[order_gate_on_qubit+3:]):
                    print(q, ind_gate, ind_gate_next)
                    if (check_rot(ind_gate_next) == True):# and (next_order_gate_on_qubit < len(qubit_gates_path[order_gate_on_qubit+3:])):
                        types.append(type_get(ind_gate_next))
                    else:
                        indices_to_compile = [on_qubit_order[q][order_gate_on_qubit+k] for k in range(len(types))]
                        if len(indices_to_compile)>3:
                            u_to_compile_db = simplified_db.loc[indices_to_compile]
                            u_to_compile_db["ind"] = (u_to_compile_db["ind"]-q) #target qubit in unitary compilation set to 0
                            u_to_compile_db["ind"] = u_to_compile_db["ind"] - translator.number_of_cnots
                            u_to_compile_db["symbol"] = None ##just to be sure it makes no interference with the compiler...

                            translator.translator_ = CirqTranslater(n_qubits=2)

                            compile_circuit, compile_circuit_db = construct_compiling_circuit(translator.translator_, u_to_compile_db)
                            minimizer = Minimizer(translator.translator_, mode="compiling", hamiltonian="Z")


                            cost, resolver, history = minimizer.minimize([compile_circuit], symbols=translator.get_symbols(compile_circuit_db))


                            u1s = u1_db(translator, 0, params=True)
                            u1s["param_value"] = -np.array(list(resolver.values()))
                            resu_comp, resu_db = translator.translator_.give_circuit(u1s,unresolved=False)

                            cc, cdb = translator.translator_.give_circuit(u_to_compile_db, unresolved=False)
                            c = cc.unitary()
                            r = resu_comp.unitary()

                            ## phase_shift if necessary
                            if np.abs(np.mean(c/r) -1) > 1:
                                u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + np.pi)

                            for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
                                simplified_db.loc[new_ind+0.1] = gate_template(translator.number_of_cnots + q + typ*translator.n_qubits,
                                                                                 param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"], 
                                                                                 trainable=True)#, symbol=symbols[new_ind])
                            for old_inds in indices_to_compile:
                                simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#
                                
                            #simplified_db = simplified_db.sort_index().reset_index(drop=True)

                    #        ## check this..!
                            ##for down_symbols in range(len(indices_to_compile)-3):
                            simplification = True
                            breakT.J. Volkoff

0 0 0


0 0 0
0 0 1
0 0 0


/home/cooper-cooper/.local/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in true_divide
/home/cooper-cooper/.local/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in true_divide


In [6]:
u1s = u1_db(translator, 0, params=True)
u1s["param_value"] = -np.array(list(resolver.values()))
resu_comp, resu_db = translator.translator_.give_circuit(u1s,unresolved=False)

cc, cdb = translator.translator_.give_circuit(u_to_compile_db, unresolved=False)
c = cc.unitary()
r = resu_comp.unitary()

## phase_shift if necessary
if np.abs(np.mean(c/r) -1) > 1:
    u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + np.pi)

for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
    simplified_db.loc[new_ind+0.1] = gate_template(translator.number_of_cnots + q + typ*translator.n_qubits,
                                                     param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"], 
                                                     trainable=True)#, symbol=symbols[new_ind])
for old_inds in indices_to_compile:
    simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#


/home/cooper-cooper/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/cooper-cooper/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


KeyError: 0

In [85]:
u1s = u1_db(translator, 0, params=True)
u1s["param_value"] = -np.array(list(resolver.values()))
resu_comp, resu_db = translator.translator_.give_circuit(u1s,unresolved=False)


ValueError: Length of values (9) does not match length of index (3)

In [ ]:

cc, cdb = translator.translator_.give_circuit(u_to_compile_db, unresolved=False)
c = cc.unitary()
r = resu_comp.unitary()

## phase_shift if necessary
if np.abs(np.mean(c/r) -1) > 1:
    u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + np.pi)

for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
    simplified_db.loc[new_ind+0.1] = gate_template(translator.number_of_cnots + q + typ*translator.n_qubits,
                                                     param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"], 
                                                     trainable=True)#, symbol=symbols[new_ind])
for old_inds in indices_to_compile:
    simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#

In [60]:

cc, cdb = translator.translator_.give_circuit(u_to_compile_db, unresolved=False)
c = cc.unitary()
r = resu_comp.unitary()


(0, 0): ───@───X───@───@───X───@───
           │   │   │   │   │   │
(0, 1): ───X───@───X───X───@───X───

In [54]:


## phase_shift if necessary
if np.abs(np.mean(c/r) -1) > 1:
    u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + np.pi)

for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
    simplified_db.loc[new_ind+0.1] = gate_template(translator.number_of_cnots + q + typ*translator.n_qubits,
                                                     param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"], 
                                                     trainable=True)#, symbol=symbols[new_ind])
for old_inds in indices_to_compile:
    simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#



[0, 1, 2, 3, 4, 5]

In [61]:
c

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [62]:
r

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [ ]:
if len(indices_to_compile)>3:
    u_to_compile_db = simplified_db.loc[indices_to_compile]
    u_to_compile_db["ind"] = (u_to_compile_db["ind"]-q) #target qubit in unitary compilation set to 0
    u_to_compile_db["ind"] = u_to_compile_db["ind"] - translator.number_of_cnots
    u_to_compile_db["symbol"] = None ##just to be sure it makes no interference with the compiler...

    translator.translator_ = CirqTranslater(n_qubits=2)

    compile_circuit, compile_circuit_db = construct_compiling_circuit(translator.translator_, u_to_compile_db)
    minimizer = Minimizer(translator.translator_, mode="compiling", hamiltonian="Z")

    cost, resolver, history = minimizer.minimize([compile_circuit], symbols=translator.get_symbols(compile_circuit_db))

In [37]:
indices_to_compile

[0, 1, 2]

In [27]:
indices_to_compile

[0, 1, 2]

In [21]:
simplified_db = simplified_db.sort_index().reset_index(drop=True)


In [22]:
circuit, circuit_db  = translator.give_circuit(circuit_db, unresolved=False)
scircuit, scircuit_db  = translator.give_circuit(simplified_db, unresolved=False)


In [23]:
scircuit.unitary()

array([[-0.84267317-0.36796345j,  0.31320255+0.23750575j],
       [-0.31320255+0.23750575j, -0.84267317+0.36796345j]])

In [24]:
circuit.unitary()

array([[-0.84267317-0.36796345j,  0.31320255+0.23750575j],
       [-0.31320255+0.23750575j, -0.84267317+0.36796345j]])

False

In [50]:
sss = simplified_db.sort_index().reset_index(drop=True)
uuco, ns = translator.give_circuit(sss, unresolved=False)
aato, os = translator.give_circuit(circuit_db,unresolved=False)

In [51]:
uuco.unitary()

array([[ 0.46061846+0.70739527j, -0.51837669-0.13677784j],
       [ 0.51837669-0.13677784j,  0.46061846-0.70739527j]])

In [52]:
aato.unitary()

array([[ 0.6803843 -0.58407108j,  0.42800579+0.11291244j],
       [-0.42800579+0.11291244j,  0.6803843 +0.58407108j]])

In [195]:
uuco

(0, 0): ───Rz(-0.421π)───Rx(0)───Rz(-1.58π)───Rz(-1.81π)───Rx(0.297π)───Rz(0.817π)───

In [196]:
indiis = np.sort(list(range(0,3*3*5,9)) + list(range(3,3*3*5,9)))

In [197]:
ll = []
for k in indiis[:-1]:
    ll.append(ns[k:(k+1)])

In [198]:
translator.give_circuit(concatenate_dbs(ll))

((0, 0): ───Rz(th_0)───Rz(th_12)───,
    ind symbol  param_value  trainable  block_id
 0    0   th_0   -13.888521       True         0
 1    0  th_12    -5.695495       True         0)

array([[-0.00512433-0.89309116j, -0.37580852-0.24724459j],
       [ 0.37580852-0.24724459j, -0.00512433+0.89309116j]])

array([[ 0.0257012 +0.89280744j,  0.37568389+0.24717591j],
       [-0.37568389+0.24717591j,  0.0257012 -0.89280744j]])

In [203]:
cost

<tf.Tensor: shape=(), dtype=float32, numpy=1.1920929e-07>